## DRF Spectacular

`drf-spectacular` is the modern standard for generating OpenAPI 3.0 schemas in Django Rest Framework. It solves many issues of previous generators (like `drf-yasg`) by supporting sophisticated type/schema generation, explicit customization, and minimal "magic".

It is primarily used to:
1.  **Generate API Documentation**: View interactive docs (Swagger UI, Redoc).
2.  **Generate Client Code**: Create TypeScript/frontend clients automatically from the schema.

### Installation & Setup

#### 1. Install the package
```bash
pipenv install drf-spectacular
```

#### 2. Configure Settings
To enable it, you must configure your `INSTALLED_APPS` and `REST_FRAMEWORK` settings in `settings.py`.

In [ ]:
# settings.py

INSTALLED_APPS = [
    # ...
    'drf_spectacular',
]

REST_FRAMEWORK = {
    # This tells DRF to use spectacular's AutoSchema for inspection
    'DEFAULT_SCHEMA_CLASS': 'drf_spectacular.openapi.AutoSchema',
}

SPECTACULAR_SETTINGS = {
    'TITLE': 'My Project API',
    'DESCRIPTION': 'Detailed description of the API',
    'VERSION': '1.0.0',
    'SERVE_INCLUDE_SCHEMA': False,
    # OTHER SETTINGS:
    # 'COMPONENT_SPLIT_REQUEST': True,
    # 'SWAGGER_UI_SETTINGS': {
    #    'deepLinking': True,
    #    'persistAuthorization': True,
    # },
}

### Exposing the Docs

You need to register the schema download view and the UI views in your `urls.py`. `drf-spectacular` provides `SpectacularAPIView` (JSON/YAML file) and two UI views: `SpectacularSwaggerView` (interactive) and `SpectacularRedocView` (cleaner, read-only).

```python
from drf_spectacular.views import SpectacularAPIView, SpectacularRedocView, SpectacularSwaggerView
from django.urls import path

urlpatterns = [
    # Generates the YAML/JSON schema file
    path('api/schema/', SpectacularAPIView.as_view(), name='schema'),
    
    # Interactive UI (Swagger)
    path('api/schema/swagger-ui/', SpectacularSwaggerView.as_view(url_name='schema'), name='swagger-ui'),
    
    # Read-only UI (Redoc)
    path('api/schema/redoc/', SpectacularRedocView.as_view(url_name='schema'), name='redoc'),
]
```

### Customization with `@extend_schema`

While `drf-spectacular` is great at auto-discovery, sometimes you need to explicitly tell it what an endpoint expects or returns, especially for custom actions or raw `APIView`s.

You use the `@extend_schema` decorator for this.

- **parameters**: List of query params, header params, or path params.
- **request**: The serializer class expected in the request body.
- **responses**: The serializer(s) returned.
- **summary/description**: Text override for the docs.
- **tags**: Grouping string.

In [ ]:
from drf_spectacular.utils import extend_schema, OpenApiParameter, OpenApiExample
from drf_spectacular.types import OpenApiTypes
from rest_framework import viewsets, status
from rest_framework.decorators import action
from rest_framework.response import Response

class UserViewSet(viewsets.ModelViewSet):
    queryset = User.objects.all()
    serializer_class = UserSerializer

    @extend_schema(
        # Describe the endpoint
        summary="Get recent users",
        description="Returns the list of 10 most recently joined users, optionally filtered by date.",
        # Query Parameters
        parameters=[
            OpenApiParameter(
                name='since',
                type=OpenApiTypes.DATE,
                location=OpenApiParameter.QUERY,
                description='Filter users joined after this date',
            ),
        ],
        # Response Definition
        responses={200: UserSerializer(many=True)}
    )
    @action(detail=False, methods=['get'])
    def recent(self, request):
        # ... logic ...
        return Response([])

### Handling Auth & Polymorphism

Spectacular automatically picks up your authentication classes from DRF settings. If you use standard classes (Session, Basic, Token, JWT), the security definitions will appear in Swagger UI automatically.

For polymorphic responses (e.g., an endpoint returning either a `Car` or a `Bike`), you can use `PolymorphicProxySerializer` from `drf_spectacular.utils` to document that the response structure varies based on the object type.

### Troubleshooting

1.  **Missing Fields**: Ensure your Serializers are using standard DRF fields. If using custom fields, you may need to write a simple extension helper.
2.  **Warnings**: Run `python manage.py spectacular --check` to see if there are missing schema definitions or warnings in your build.